In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import (
    Conv1D, BatchNormalization, Activation,
    MaxPooling1D, Dropout, Flatten, Dense
)
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Load data
data = np.load('stroke_peak_data.npz')
X = data['X']
y = data['y']

print(f"Total samples: {X.shape[0]}, each window shape: {X.shape[1:]}")
print(f"Labels shape: {y.shape}")

Total samples: 658, each window shape: (64, 6)
Labels shape: (658,)


In [5]:
# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

num_classes = len(set(y_train))

print("After split:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_test : {X_test.shape},  y_test : {y_test.shape}")

After split:
  X_train: (526, 64, 6), y_train: (526,)
  X_test : (132, 64, 6),  y_test : (132,)


In [10]:
# Conv1D Model
def build_model():
    model = Sequential([
        Input(shape=(64, 6)),

        Conv1D(64, kernel_size=3, padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.2),

        Conv1D(128, kernel_size=3, padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model
#model.summary()

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
val_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
    print(f"\n--- Fold {fold+1} ---")
    X_tr, X_va = X_train[train_idx], X_train[val_idx]
    y_tr, y_va = y_train[train_idx], y_train[val_idx]

    # 4. Build a fresh model and callbacks for each fold
    model = build_model()
    early = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

    # 5. Train
    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=256,
        batch_size=16,
        callbacks=[early],
        verbose=0
    )

    # 6. Evaluate on this fold’s validation set
    val_loss, val_acc = model.evaluate(X_va, y_va, verbose=0)
    print(f"Fold {fold+1} val accuracy: {val_acc:.4f}")
    val_accuracies.append(val_acc)

# 7. Aggregate results
val_accuracies = np.array(val_accuracies)
print(f"\nMean val accuracy: {val_accuracies.mean():.4f} ± {val_accuracies.std():.4f}")

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")


--- Fold 1 ---
Fold 1 val accuracy: 0.6792

--- Fold 2 ---
Fold 2 val accuracy: 0.6286

--- Fold 3 ---
Fold 3 val accuracy: 0.6476

--- Fold 4 ---
Fold 4 val accuracy: 0.6667

--- Fold 5 ---
Fold 5 val accuracy: 0.6286

Mean val accuracy: 0.6501 ± 0.0203
Test accuracy: 0.591


In [9]:
# Training
early = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=256,
    batch_size=16,
    callbacks=[early]
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1/256
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.1263 - loss: 2.6427 - val_accuracy: 0.1288 - val_loss: 2.4839
Epoch 2/256
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.4307 - loss: 1.7071 - val_accuracy: 0.1742 - val_loss: 2.4842
Epoch 3/256
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.6103 - loss: 1.1864 - val_accuracy: 0.2955 - val_loss: 2.0941
Epoch 4/256
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7686 - loss: 0.7752 - val_accuracy: 0.3864 - val_loss: 1.7910
Epoch 5/256
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8515 - loss: 0.4707 - val_accuracy: 0.4015 - val_loss: 1.6998
Epoch 6/256
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9624 - loss: 0.2479 - val_accuracy: 0.4394 - val_loss: 1.7198
Epoch 7/256
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9662 - loss: 0.1914 - val_accuracy: 0.4773 - val_loss: 1.5890
Epoch 8/256
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9789 - loss: 0.1323 - val_accuracy: 0.

In [ ]:
# Convert to TFLite
def representative_dataset():
    for input_value in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(100):
        # ensure dtype matches your input dtype (float32 here)
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset

# For full integer quantization (weights + activations)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type  = tf.uint8   # or tf.int8
converter.inference_output_type = tf.uint8   # match input type

tflite_model = converter.convert()
open('conv1d_model.tflite', 'wb').write(tflite_model)
print("TFLite model written to conv1d_model.tflite")